In [1]:
import numpy as np
from sympy import *
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import display, Math, Latex
init_printing(use_latex='mathjax')

In [2]:
def print_eq(eq, *exprs):
    if type(eq) == str:
        eq = Symbol(eq)
    out = '$$ {} ' + '= {}' * len(exprs) + '$$'
    exprs=list(exprs)
    for i in range(len(exprs)):
        if type(exprs[i]) != str:
            exprs[i] = latex(exprs[i])
    display(Latex(out.format(latex(eq), *exprs)))

def print_system(*exprs, collab=False): 
    '''
    collab говнюк не хочет нормально системы уравнений отоброжать, 
    если вы в Jupyter, то замените в аргументе collab True на False'''
    exprs=list(exprs)
    for i in range(len(exprs)):
        if type(exprs[i]) != str:
            exprs[i] = latex(exprs[i])
    exprs_latex = r'\\'.join(exprs)
    if collab:
        system = r'Начало системы\\' + exprs_latex + r'\\Конец_системы\\'
    else:
        system = \
        r'''\begin{equation*}
    \begin{cases}''' \
        + exprs_latex + \
        r'''\end{cases}
    \end{equation*}
        '''

    print_eq(system)
    
def to_s(i):
    return Symbol(str(i))

def pmatrix(array):
    if type(array) != Matrix:
        array = Matrix(array)
    return latex(array)[6:-7].replace('matrix', 'pmatrix') + ''

In [3]:
def get_acc(X_cur, X_prev, lat=False):
    acc = 0
    acc_lat = ''
    for i in range(len(X_cur)):
        acc += (X_cur[i] - X_prev[i]) ** 2
        acc_lat += f'({X_cur[i]} - {X_prev[i]})^{{2}}+'
    return round(acc, 12), acc_lat[:-1]

In [4]:
sym = symbols('x, y, z, c')

In [5]:
A = np.array([
    [10, 1, 1, 2],
    [-1, 19, 2, -2],
    [-2, -1, 20, -1],
    [1, 2, 1, 11]
])
b = [10, 60, 90, 60]
ACC = 0.001
print_eq('A', pmatrix(A))
print_eq('b', pmatrix(b))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [6]:
exprs = []
exprs_lat = []
for i in range(len(A)):
    expr_lat = ''
    expr = 0
    for j in range(len(A[i])):
        expr += sym[j] * A[i][j]
        comp = latex(sym[j] * A[i][j])
        if not comp.startswith('-') and j != 0:
            comp = '+' + comp
        expr_lat += comp
    expr_lat += f'={b[i]}' 
    exprs_lat += [expr_lat]
    exprs += [expr]
print_system(*exprs_lat)

<IPython.core.display.Latex object>

In [7]:
new_values = []
coefs_expressed = []
new_exprs_lat = []
for i in range(len(A)):
    el_express = A[i][i]
    if el_express % 10 > 5:
        expressed = el_express // 10 * 10 + 10
    else:
        expressed = el_express - el_express % 10
    expr_lat = f'{expressed}{sym[i]}='
    coefs_expressed += [expressed]
    new_values += [[]]
    for j in range(len(A[i])):
        if j != i:
            new_values[-1].append(-A[i][j])
        else:
            new_values[-1].append(expressed - el_express)
            
        comp = latex(sym[j] * new_values[-1][-1])
        if not comp.startswith('-') and j != 0:
            comp = '+' + comp
        expr_lat += comp
    new_values[-1].append(b[i])
    expr_lat += f'+{b[i]}' if b[i] > 0 else str(b[i]) 
    new_exprs_lat += [expr_lat]
print_system(*new_exprs_lat)

<IPython.core.display.Latex object>

In [8]:
div_exprs = []
C = np.array(new_values, dtype='float64')
for i in range(len(C)):
    C[i] = C[i] / coefs_expressed[i]
F = np.array(C[:, -1])
C = np.array(C[:, :-1])
for i in range(len(C)):
    div_expr = f'{sym[i]}='
    for j in range(len(C[i])): 
        comp = latex(sym[j] * C[i][j])
        if not comp.startswith('-') and j != 0:
            comp = '+' + comp
        div_expr += comp
    div_expr += f'+{F[i]}' if b[i] > 0 else str(b[i]) 
    div_exprs += [div_expr]
print_system(*div_exprs)

<IPython.core.display.Latex object>

In [9]:
C_abs = np.vectorize(abs)(C)
C1 = C_abs.sum(axis=0)
C_inf = C_abs.sum(axis=1)
max_str = f'max\{{{";".join(C1.astype(str))}\}}={max(C1)}'
print_eq('||C||_1', max_str)
max_str = f'max\{{{";".join(C_inf.astype(str))}\}}={max(C_inf)}'
print_eq('||C||_{\infty}', max_str)
C2 = 0
C2_value = 0 
for row in C_abs:
    for el in row:
        if el != 0:
            C2 += to_s(el) ** 2
            C2_value += el ** 2
C2_value = round(C2_value, 8)
print_eq('||C||_2', sqrt(C2), latex(sqrt(to_s(C2_value))) + f'\\approx {round(sqrt(C2_value), 3)}')
if min(max(C1), max(C_inf), C2_value) < 1:
    print_eq('||C|| < 1')
else:
    print_eq('Беда')

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [10]:
X = [Matrix([0] * len(b))]
C_mat = Matrix(C)
F_mat = Matrix(F)
xyz_m = Matrix(sym)
i = 0
print_eq(f'X^{{({i})}}', pmatrix(X[0]))
while True:
    i += 1
    X.append(C_mat * X[-1] + F_mat)
    print_eq(pmatrix(xyz_m) + f'^{{({i})}}', pmatrix(C_mat) + pmatrix(X[-2]) + '+' + pmatrix(F_mat), pmatrix(X[-1]))
    acc = get_acc(X[-1], X[-2])[0]
    if acc < ACC:
        print_eq(get_acc(X[-1], X[-2])[1], latex(acc) + f'< {ACC}')
        print_eq(f'X^{{({i})}}', pmatrix(X[-1]), r'\bar X')
        break
    else:
        print_eq(f'X^{{({i})}}', pmatrix(X[-1]))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [11]:
print('Ответ:')
print_eq(r'\bar X', pmatrix(X[-1]))

Ответ:


<IPython.core.display.Latex object>